In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
from cs285.envs.pendulum.pendulum_env import PendulumEnv
from cs285.envs.dt_sampler import ConstantSampler
from cs285.infrastructure.replay_buffer import ReplayBufferTrajectories
from cs285.infrastructure.utils import sample_n_trajectories, RandomPolicy
from cs285.agents.ode_agent_jax import ODEAgent_jax
from typing import Callable, Optional, Tuple, Sequence
import numpy as np
import torch.nn as nn
import torch
import gym
from cs285.infrastructure import pytorch_util as ptu
from torchdiffeq import odeint
from tqdm import trange
import jax
import jax.numpy as jnp
import equinox as eqx
import diffrax
from diffrax import diffeqsolve, Dopri5
import optax

In [2]:
key = jax.random.PRNGKey(0)
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)
device = "gpu" if torch.cuda.is_available() else "cpu"

cpu


In [3]:
dt_sampler = ConstantSampler(dt=0.05)
env = PendulumEnv(
    dt_sampler=dt_sampler
)
mb_agent_jax = ODEAgent_jax(
    env=env,
    hidden_size=128,
    num_layers=4,
    ensemble_size=10,
    train_timestep=0.005,
    mpc_horizon_steps=100,
    mpc_timestep=0.005,
    mpc_strategy="random",
    mpc_num_action_sequences=10,
    key=key
)
replay_buffer = ReplayBufferTrajectories(seed=42)
trajs, _ = sample_n_trajectories(env, RandomPolicy(env=env), ntraj=10, max_length=200)
replay_buffer.add_rollouts(trajs)

100%|██████████| 10/10 [00:00<00:00, 47.00it/s]


In [10]:
for n in trange(2):
    for i in range(mb_agent_jax.ensemble_size):
        traj = replay_buffer.sample_rollout()
        loss = mb_agent_jax.update(i, traj["observations"], traj["actions"], jnp.cumsum(traj["dts"]), discount=0.9)
# TODO: change the implementation to allow sampling batched rollouts

100%|██████████| 2/2 [00:33<00:00, 16.97s/it]


In [12]:
loss

0.15550266206264496

In [4]:
batch_size = 64
for n in trange(2):
    for i in range(mb_agent_jax.ensemble_size):
        obs, acs, times = jnp.empty(shape=(batch_size, 201, 3)), jnp.empty(shape=(batch_size, 201, 1)), jnp.empty(shape=(batch_size, 201))
        for m in range(batch_size):
            traj = replay_buffer.sample_rollout()
            obs.at[m].set(traj["observations"])
            acs.at[m].set(traj["actions"])
            times.at[m].set(jnp.cumsum(traj["dts"]))
        loss = mb_agent_jax.batched_update_gd(i=i, obs=obs, acs=acs, times=times, discount=0.9)


  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:39<00:00, 19.80s/it]


In [5]:
def func():
    batch_size = 64
    for n in trange(2):
        for i in range(mb_agent_jax.ensemble_size):
            obs, acs, times = jnp.empty(shape=(batch_size, 201, 3)), jnp.empty(shape=(batch_size, 201, 1)), jnp.empty(shape=(batch_size, 201))
            for m in range(batch_size):
                traj = replay_buffer.sample_rollout()
                obs.at[m].set(traj["observations"]) # takes 1% of time
                acs.at[m].set(traj["actions"]) # takes 1% of time
                times.at[m].set(jnp.cumsum(traj["dts"])) # takes 1% of time
            loss = mb_agent_jax.batched_update_gd(i=i, obs=obs, acs=acs, times=times, discount=0.9)

In [6]:
%lprun -f func func()

100%|██████████| 2/2 [00:42<00:00, 21.12s/it]


Timer unit: 1e-09 s

Total time: 42.2433 s
File: /var/folders/nd/k3zbj8ys0v544bkn7b8pjnf80000gn/T/ipykernel_8554/2015805227.py
Function: func at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def func():
     2         1          0.0      0.0      0.0      batch_size = 64
     3         3   11058000.0    4e+06      0.0      for n in trange(2):
     4        22      15000.0    681.8      0.0          for i in range(mb_agent_jax.ensemble_size):
     5        20    6805000.0 340250.0      0.0              obs, acs, times = jnp.empty(shape=(batch_size, 201, 3)), jnp.empty(shape=(batch_size, 201, 1)), jnp.empty(shape=(batch_size, 201))
     6      1300     291000.0    223.8      0.0              for m in range(batch_size):
     7      1280    6186000.0   4832.8      0.0                  traj = replay_buffer.sample_rollout()
     8      1280  540241000.0 422063.3      1.3                  obs.at[m].set(traj["observation